In [1]:
import os
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence, to_categorical

print(tf.test.is_gpu_available())

max_frame = 40

True


https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://pytorch.org/docs/stable/data.html

https://github.com/kenshohara/3D-ResNets-PyTorch

In [2]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)
labels = {}
with open("data/labels.json", "r") as file:
    labels = json.load(file)

In [3]:
class NTUSequence(Sequence):
    
    def __init__(self, list_IDs, labels, path, batch_size=32, max_frame=max_frame, one_hot=True):
        self.x = list_IDs
        self.y = labels
        self.batch_size = batch_size
        self.max_frame = max_frame
        self.path = path
        if one_hot:
            self.one_hot_encode()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = np.array([self.y[ID] for ID in batch_x])
        
        X = [np.load(self.path + ID + '.npy')[:, :, :, np.newaxis] for ID in batch_x]
        
        if not(self.max_frame is None):
            X = np.stack(
                [x[:max_frame] if x.shape[0] >= max_frame else np.pad(x, ((0, max_frame-x.shape[0]),(0,0),(0,0),(0,0)), "constant") for x in X]
            )     
        
        return X, batch_y

    def one_hot_encode(self):
        labels = [val for val in self.y.values()]
        ids = [key for key in self.y.keys()]
        labels = to_categorical(labels)
        self.y = {ID: label for ID, label in zip(ids, labels)}

In [4]:
trainset = NTUSequence(list_IDs["train"], labels, path="data/processed/train/")
testset = NTUSequence(list_IDs["validation"], labels, path="data/processed/test/")

# Conv3D 

In [5]:
conv3d = models.Sequential([
    layers.Conv3D(64, 3, activation="relu", padding="same", input_shape=(max_frame, 25, 25, 1)),
    layers.Conv3D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Conv3D(128, 3, activation="relu", padding="same"),
    layers.Conv3D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Conv3D(256, 3, activation="relu", padding="same"),
    layers.Conv3D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling3D((2,2,2)),
    
    layers.Flatten(),
    #layers.Dense(1, activation="relu"),
    layers.Dense(49, activation="softmax")
])

conv3d.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 40, 25, 25, 64)    1792      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 40, 25, 25, 64)    110656    
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 20, 12, 12, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 20, 12, 12, 128)   221312    
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 20, 12, 12, 128)   442496    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 6, 6, 128)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 10, 6, 6, 256)     8

In [6]:
conv3d.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
history = conv3d.fit_generator(trainset, 
                     epochs=20)

Epoch 1/20
1157/1157 [==============================] - 1479s 1s/step - loss: 3.8929 - accuracy: 0.0197
Epoch 2/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0201
Epoch 3/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0199
Epoch 4/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0196
Epoch 5/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0203
Epoch 6/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0191
Epoch 7/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0203
Epoch 8/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0200
Epoch 9/20
1157/1157 [==============================] - 1470s 1s/step - loss: 3.8921 - accuracy: 0.0196
Epoch 10/20
1157/1157 [==============================] - 1470s 1

# Pytorch ?

In [ ]:
class NTUDataset(data.Dataset):
    def __init__(self, list_IDs, labels, path, max_frame=max_frame):
            self.max_frame = max_frame
            self.path = path
            self.labels = labels
            self.list_IDs = list_IDs

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]
        array = np.load(self.path + ID + '.npy')
        if not(self.max_frame is None):
            mid_frame = array.shape[0] // 2
            array = array[mid_frame-self.max_frame//2:mid_frame+self.max_frame//2]
        # Load data and get label
        X = torch.from_numpy(array)
        y = self.labels[ID]

        return X, y
    
trainset = NTUDataset(list_IDs["train"], labels, path="data/processed/train/")
testset = NTUDataset(list_IDs["validation"], labels, path="data/processed/test/")

train_gen = data.DataLoader(trainset, **dataloader_config)
test_gen = data.DataLoader(testset, **dataloader_config)

In [ ]:
class SpatialClassifier(nn.Module):
    
    def __init__(self):
        super(SpatialClassifier, self).__init__()
        self.conv11 = nn.Conv3d(max_frame, 64, 3)
        self.conv12 = nn.Conv3d(64, 64, 3)
        
        self.conv21 = nn.Conv3d(128, 128, 3)
        self.conv22 = nn.Conv3d(128, 128, 3)

        self.conv31 = nn.Conv3d(256, 256, 3)
        self.conv32 = nn.Conv3d(256, 256, 3)
        
        self.fc = nn.Linear(256*6*6, 11)
        
    def forward(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool3d(2)
        
        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool3d(2)
        
        x = F.relu(self.conv31(x))
        x = F.relu(self.conv32(x))
        x = F.max_pool3d(2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
net = SpatialClassifier()
net.to(device)
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
running_loss = 0.0
for i, data in enumerate(train_gen, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')